In [1]:
# importing all the required modules

from importlib import reload
import nltk
from bs4 import BeautifulSoup
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
import os
import errno
import string
from nltk.corpus import reuters
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk import FreqDist
from nltk.text import TextCollection
import collections
import numpy as np
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from prettytable import PrettyTable
import time
from datetime import timedelta
from sklearn.metrics import recall_score,precision_score,average_precision_score,f1_score
from sklearn.metrics import accuracy_score,roc_auc_score,mean_absolute_error,cohen_kappa_score
from sklearn.preprocessing import label_binarize,LabelEncoder
from imblearn.metrics import geometric_mean_score
from sklearn import metrics
from sklearn.metrics.cluster import homogeneity_score,completeness_score
import statistics
import math
import sklearn.metrics 
from sklearn.model_selection import KFold,StratifiedKFold
import csv
from sklearn.model_selection import train_test_split
from collections import Counter
import math
from scipy.spatial import distance
import statistics
from scipy.stats import pearsonr,entropy,mode
from scipy.io import arff
from sklearn.metrics import pairwise_distances
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import NearestNeighbors
from sklearn.datasets import load_iris
import warnings
import random
from statistics import mode
random.seed(0)
warnings.filterwarnings('ignore')
import os 
os.environ['R_HOME'] = 'C:\Program Files\R\R-4.3.2'
import rpy2.robjects as robjects
import rpy2.robjects.numpy2ri
rpy2.robjects.numpy2ri.activate()


def loadNonIRDatasets(dataset):
    documents,labels=[],[]
    fname=dataset+'.csv'
    with open(fname, 'r') as file:
        reader = csv.reader(file, delimiter = ',')
        for row in reader:
            if len(row)>0:
                if '?' in row:
                    continue
                doc=row[0:len(row)-2]
                try:
                    doc=[abs(float(d)) for d in doc]
                except:
                    continue
                documents.append(doc)
                #print(row)
                labels.append(row[len(row)-1])

    return documents,labels

def Cosine(a, b):#distance
    return distance.cosine(a,b)

def Euclidean(a, b):#distance
    return distance.euclidean(a,b)

def most_common(lst):
    return max(set(lst), key=lst.count)
def PrintDetails(metric="smtp",measure= "Accuracy",Arr=None,kList=[5,15,30,45],train_time=0,test_time=0,std=[]):
    x = PrettyTable()
    #kList=[1,3,5,9,15,30,45]#,70,90,120]
    #if std==[]:
    if len(std)==0:
        x.field_names = ["k",measure,"Train Time","Test time","Total Time"]
    else:
        x.field_names = ["Split",measure,"std","Train Time","Test time","Total Time"]
        
    print("metric",metric)
    print("measure",measure)
    tables.write(("metric\t"+str(metric)+"\n").encode())
    tables.write(("measure\t"+str(measure)+"\n").encode())
    average=0
    total_time=np.add(train_time,test_time)
    for i in range(0,len(Arr)):
        print(train_time[i])
        tr_time=str(timedelta(seconds=(train_time[i])))
        ts_time=str(timedelta(seconds=(test_time[i])))
        
        tot_time=str(timedelta(seconds=(total_time[i])))
        if len(std)==0:
            x.add_row([str(kList[i]),round(Arr[i],4),tr_time,ts_time,tot_time]) 
        else:
            x.add_row([str(kList[i]),round(Arr[i],4),round(std[i],4),tr_time,ts_time,tot_time]) 
            
        average+=Arr[i]
    if len(std)==0:
        x.add_row(["Average",round((average/len(Arr)),4),str(timedelta(seconds=np.mean(train_time))),
                   str(timedelta(seconds=np.mean(test_time))),str(timedelta(seconds=np.mean(total_time)))]) 
        x.add_row(["Std Dev.",round(np.std(Arr),4),np.std(train_time),
                   np.std(test_time),np.std(total_time)])   
    else:
        x.add_row(["Average",round((average/len(Arr)),4),round(np.mean(std),4),str(timedelta(seconds=np.mean(train_time))),
                   str(timedelta(seconds=np.mean(test_time))),str(timedelta(seconds=np.mean(total_time)))]) 
        x.add_row(["Std Dev.",round(np.std(Arr),4),round(np.std(std),4),
                   round(np.std(train_time),10),
                   round(np.std(test_time),10),
                   round(np.std(total_time),10)])
        
    x.padding_width =1
    tables.write(str(x).encode())
    print(x)

def findDI(train_data,train_labels,metric,k):
    clss=set(train_labels)
    func=globals()[metric]
    radius={}
    th=0.5
    for cl in clss:
        indices=[i for i in range(0,len(train_labels)) if train_labels[i]==cl]
        elements=train_data[indices,:]
        distMat=pairwise_distances(elements,elements,metric=func)
        radius[cl]=np.sum(distMat)/(distMat.shape[0]+distMat.shape[1])
    weights=[]
   
    classifier = NearestNeighbors(n_neighbors=k,metric=func)
    if k>len(train_labels):
         classifier = NearestNeighbors(n_neighbors=len(train_labels),metric=func)
        
    classifier.fit(train_data,train_labels)
    distances,neighVals=classifier.kneighbors(train_data,return_distance=True)
    print(neighVals.shape)
    for i in range(0,train_data.shape[0]):
        r=radius[train_labels[i]]
        ID=0
        neighs=np.array(neighVals[i,:],dtype=int)
        #print(neighs)
        neighLabels = [train_labels[i] for i in neighs]
           
        #neighLabels = [train_labels[i] for i in int(neighVals[i,:])]
        val=0
        c=1
        for nn,neigh in enumerate(neighLabels):
            if distances[i,nn]<r:
                c=c+1
                if neigh==train_labels[i]:
                    val=val+1
        ID=(val/c)
        ##############find weights using validity ###############
        #w=np.sum(val*(1/(distances[i]+alpha)))
        weights.append(ID)
    return weights

def classify(k=3,metric="euclidean",termOccurance=None, docOccurance=None):
    accuracy=[]
    precision=[]
    Recall=[]
    fMeasure=[]
    mprecision=[]
    mRecall=[]
    mfMeasure=[]
    roc=[]
    y_pred=[]
    mae=[]
    kappa=[]
    averageMeanPrecison=[]
    func=globals()[metric]
    kList=[5,15,30,45]
    theta=1
    var=np.zeros(train_data.shape[1])
    train_time=[]
    test_time=[]
    M=5
    alpha=0.5
    for k in kList:
        time1=time.time()
        robjects.r.source('kCNN.R')
        time2=time.time()
        pred,temp = robjects.r['kcnn'](train=train_data, test=test_data,cl=train_labels, k = k)
        pred = [eval(ii) for ii in pred]      
        y_pred.append(pred)
        time3=time.time()
        train_time.append(time2-time1)
        test_time.append(time3-time2)
 
   
    class_dict={}
    classes=list(set(labels))
    for i in range(0,len(classes)):
        class_dict[classes[i]]=i
    print(class_dict)
    num_testLabels=[]
    for i in range(0,len(test_labels)):
        num_testLabels.append(class_dict[test_labels[i]])
       
    for i in range(0,len(y_pred)):
        test_labels1=list(test_labels)
        #Accuracy
       
        accuracy.append(accuracy_score(test_labels, y_pred[i]))
        
        #Precison
        precision.append(precision_score(test_labels, y_pred[i],average='macro'))
         
        #Recall
        Recall.append(recall_score(test_labels, y_pred[i],average='macro'))
           
        #f measure
        fMeasure.append(f1_score(test_labels, y_pred[i],average='macro'))
        
        #Precison
        mprecision.append(precision_score(test_labels, y_pred[i],average='micro'))
         
        #Recall
        mRecall.append(recall_score(test_labels, y_pred[i],average='micro'))
           
        #f measure
        mfMeasure.append(f1_score(test_labels, y_pred[i],average='micro'))
        
        kappa.append(cohen_kappa_score(test_labels, y_pred[i]))
      
        test = label_binarize(test_labels1, classes=categories).reshape((-1))
        pred = label_binarize( y_pred[i], classes=categories).reshape((-1))
        
        #roc
        try:
            roc.append(roc_auc_score(test,pred))
        except:
            roc.append(-1)
            
        #Average Mean Precision
        #y_val_true, val_pred = y_val_true.reshape((-1)), val_pred.reshape((-1))
        averageMeanPrecison.append(average_precision_score(test, pred))
        #MAE
        num_pred=[]
        for j in range(0,len(y_pred[i])):
            num_pred.append(class_dict[y_pred[i][j]])
        mae.append(mean_absolute_error(num_testLabels, num_pred))
        
        #PrintDetails(metric=metric,time=str(timedelta(seconds=(time2-time1))),measure="Average Mean Precison",Arr=averageMeanPrecison)


    
   #Accuracy
    PrintDetails(metric=metric,measure="Accuracy",Arr=accuracy,train_time=train_time,test_time=test_time)
    
    #Precison
    PrintDetails(metric=metric,measure="Precision",Arr=precision,train_time=train_time,test_time=test_time)
    #Precison
    PrintDetails(metric=metric,measure="micro Precision",Arr=mprecision,train_time=train_time,test_time=test_time)
    #Recall
    PrintDetails(metric=metric,measure="Recall",Arr=Recall,train_time=train_time,test_time=test_time)
     #Recall
    PrintDetails(metric=metric,measure="micro Recall",Arr=mRecall,train_time=train_time,test_time=test_time)
    #f measure
    PrintDetails(metric=metric,measure="F Measure",Arr=fMeasure,train_time=train_time,test_time=test_time)
    
    #f measure
    PrintDetails(metric=metric,measure="micro F Measure",Arr=mfMeasure,train_time=train_time,test_time=test_time)
    
    #roc
    PrintDetails(metric=metric,measure="ROC",Arr=roc,train_time=train_time,test_time=test_time)
    #Average Mean Precision
    PrintDetails(metric=metric,measure="Average Mean Precison",Arr=averageMeanPrecison,train_time=train_time,test_time=test_time)
    #MAE
    PrintDetails(metric=metric,measure="MAE",Arr=mae,train_time=train_time,test_time=test_time)
  
    accuracy_avg.append(accuracy)
    precision_avg.append(precision)
    recall_avg.append(Recall)
    macroFMeasure_avg.append(fMeasure)
    mprecision_avg.append(mprecision)
    mrecall_avg.append(mRecall)
    mFMeasure_avg.append(mfMeasure)
    roc_avg.append(roc)
    AMP_avg.append(averageMeanPrecison)
    mae_avg.append(mae)
    avg_test_time.append(test_time)
    avg_train_time.append(train_time)
    
import glob
datasets=[]
for file_name in glob.glob("Latest Datasets1\\"+'*.csv'):
    file=os.path.split(file_name)[1]
    datasets.append(os.path.splitext(file)[0])

    
for dataset in datasets:
    fname="Latest Datasets1\\"+dataset
    print(fname.encode())
    arr,labels=loadNonIRDatasets(fname)
    arr=np.array(arr)
    labels=np.array(labels)
    labels=list(labels)
    le = LabelEncoder()
    labels=le.fit_transform(labels)
    categories=list(set(labels))
    measures=["Euclidean"]
    global w
    w=0
    global Weights
    for met in measures:
        n_split=1
        labels=np.array(labels)
        path = "CKNN\\April Task1\\Latest Datasets"
        if not os.path.exists(path):
            os.makedirs(path)
        fname=path+'\\table_'+dataset+'_'+met+'_CKNN.txt'
        tables = open(fname, 'wb')
        k_splits=10
        accuracy_avg=[]
        precision_avg=[]
        recall_avg=[]
        macroFMeasure_avg=[]
        mprecision_avg=[]
        mrecall_avg=[]
        mFMeasure_avg=[]
        roc_avg=[]
        AMP_avg=[]
        mae_avg=[]
        avg_test_time=[]
        avg_train_time=[]
        kf = StratifiedKFold(n_splits=k_splits)
        for trn_ind, tst_ind in kf.split(arr,labels):
            xTrain, xTest = arr[trn_ind], arr[tst_ind]
            yTrain, yTest = labels[trn_ind], labels[tst_ind]
            train_data=np.array(xTrain)
            test_data=np.array(xTest)
            train_labels=np.array(yTrain)
            test_labels=np.array(yTest)
            allData=arr
            termOcc=[]
            docOcc=[]
            tables.write("\n*************".encode())
            tables.write((" Split\t"+str(n_split)).encode())
            tables.write(" ***********\n".encode())
            n_split=n_split+1
            k=1
            print("###############")
            Weights=np.ones(train_data.shape[0])
            classify(k=k,metric=met,termOccurance=termOcc,docOccurance=docOcc)
        list1=[5,15,30,45]
        tables.write("\n************* Average Results ***********\n".encode())
        acc_std=np.std(np.array(accuracy_avg),axis=0)
        accuracy_avg=np.mean(np.array(accuracy_avg),axis=0)
        
        pr_std=np.std(np.array(precision_avg),axis=0)
        precision_avg=np.mean(np.array(precision_avg),axis=0)
        
        mpr_std=np.std(np.array(mprecision_avg),axis=0)
        mprecision_avg=np.mean(np.array(mprecision_avg),axis=0)
        
        rcl_std=np.std(np.array(recall_avg),axis=0)
        recall_avg=np.mean(np.array(recall_avg),axis=0)
        
        mrcl_std=np.std(np.array(mrecall_avg),axis=0)
        mrecall_avg=np.mean(np.array(mrecall_avg),axis=0)
        
        F_std=np.std(np.array(macroFMeasure_avg),axis=0)
        macroFMeasure_avg=np.mean(np.array(macroFMeasure_avg),axis=0)
        
        mF_std=np.std(np.array(mFMeasure_avg),axis=0)
        mFMeasure_avg=np.mean(np.array(mFMeasure_avg),axis=0)
        
        roc_std=np.std(np.array(roc_avg),axis=0)
        roc_avg=np.mean(np.array(roc_avg),axis=0)
        
        amp_std=np.std(np.array(AMP_avg),axis=0)
        AMP_avg=np.mean(np.array(AMP_avg),axis=0)
        
        mae_std=np.std(np.array(mae_avg),axis=0)
        mae_avg=np.mean(np.array(mae_avg),axis=0)
        
        avg_train_time=np.mean(np.array(avg_train_time),axis=0)
        avg_test_time=np.mean(np.array(avg_test_time),axis=0)
        #Accuracy
        PrintDetails(metric=met,measure="Accuracy",Arr=accuracy_avg,kList=list1,train_time=avg_train_time,test_time=avg_test_time,std=acc_std)
        
        #Precision
        PrintDetails(metric=met,measure="Precision",Arr=precision_avg,kList=list1,train_time=avg_train_time,test_time=avg_test_time,std=pr_std)
        
        #Precision
        PrintDetails(metric=met,measure="Micro Precision",Arr=mprecision_avg,kList=list1,train_time=avg_train_time,test_time=avg_test_time,std=mpr_std)


        #Recall
        PrintDetails(metric=met,measure="Recall",Arr=recall_avg,kList=list1,train_time=avg_train_time,test_time=avg_test_time,std=rcl_std)
        
         #Recall
        PrintDetails(metric=met,measure="Micro Recall",Arr=mrecall_avg,kList=list1,train_time=avg_train_time,test_time=avg_test_time,std=mrcl_std)


        #f measure
        PrintDetails(metric=met,measure="F Measure",Arr=macroFMeasure_avg,kList=list1,train_time=avg_train_time,test_time=avg_test_time,std=F_std)
        
       
       
        #f measure
        PrintDetails(metric=met,measure="Micro F Measure",Arr=mFMeasure_avg,kList=list1,train_time=avg_train_time,test_time=avg_test_time,std=mF_std)


        #roc
        PrintDetails(metric=met,measure="ROC",Arr=roc_avg,kList=list1,train_time=avg_train_time,test_time=avg_test_time,std=roc_std)

        #average Mean precision
        PrintDetails(metric=met,measure="AMP",Arr=AMP_avg,kList=list1,train_time=avg_train_time,test_time=avg_test_time,std=amp_std)
         
        #MAE
        PrintDetails(metric=met,measure="MAE",Arr=mae_avg,kList=list1,train_time=avg_train_time,test_time=avg_test_time,std=mae_std)
        
        tables.close()
        print("###############")





C:\Users\Fix-Dell\anaconda3\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


b'Latest Datasets1\\banknote'
###############
{0: 0, 1: 1}
metric Euclidean
measure Accuracy
0.022937536239624023
0.002023458480834961
0.0019915103912353516
0.003962516784667969
+----------+----------+----------------------+----------------------+---------------------+
|    k     | Accuracy |      Train Time      |      Test time       |      Total Time     |
+----------+----------+----------------------+----------------------+---------------------+
|    5     |  0.9565  |    0:00:00.022938    |    0:00:00.613871    |    0:00:00.636809   |
|    15    |  0.9348  |    0:00:00.002023    |    0:00:00.550530    |    0:00:00.552554   |
|    30    |  0.9275  |    0:00:00.001992    |    0:00:00.565730    |    0:00:00.567721   |
|    45    |  0.9275  |    0:00:00.003963    |    0:00:00.584465    |    0:00:00.588428   |
| Average  |  0.9366  |    0:00:00.007729    |    0:00:00.578649    |    0:00:00.586378   |
| Std Dev. |  0.0119  | 0.008817000073179112 | 0.023622023283222056 | 0.03177898170079

{0: 0, 1: 1}
metric Euclidean
measure Accuracy
0.002027750015258789
0.002979755401611328
0.0029916763305664062
0.002986431121826172
+----------+----------+------------------------+----------------------+----------------------+
|    k     | Accuracy |       Train Time       |      Test time       |      Total Time      |
+----------+----------+------------------------+----------------------+----------------------+
|    5     |  0.942   |     0:00:00.002028     |    0:00:00.562702    |    0:00:00.564730    |
|    15    |  0.9493  |     0:00:00.002980     |    0:00:00.522608    |    0:00:00.525587    |
|    30    |  0.9348  |     0:00:00.002992     |    0:00:00.555089    |    0:00:00.558081    |
|    45    |  0.9348  |     0:00:00.002986     |    0:00:00.557513    |    0:00:00.560500    |
| Average  |  0.9402  |     0:00:00.002746     |    0:00:00.549478    |    0:00:00.552225    |
| Std Dev. |  0.006   | 0.00041493612814792773 | 0.015755607672571444 | 0.015562035518103354 |
+----------+-

{0: 0, 1: 1}
metric Euclidean
measure Accuracy
0.0020291805267333984
0.0029571056365966797
0.004008769989013672
0.0019621849060058594
+----------+----------+-----------------------+----------------------+----------------------+
|    k     | Accuracy |       Train Time      |      Test time       |      Total Time      |
+----------+----------+-----------------------+----------------------+----------------------+
|    5     |  0.9416  |     0:00:00.002029    |    0:00:00.543620    |    0:00:00.545650    |
|    15    |  0.8905  |     0:00:00.002957    |    0:00:00.538354    |    0:00:00.541311    |
|    30    |  0.8905  |     0:00:00.004009    |    0:00:00.551524    |    0:00:00.555532    |
|    45    |  0.8978  |     0:00:00.001962    |    0:00:00.575460    |    0:00:00.577422    |
| Average  |  0.9051  |     0:00:00.002739    |    0:00:00.552240    |    0:00:00.554979    |
| Std Dev. |  0.0213  | 0.0008317407774577618 | 0.014202161673128873 | 0.013945050049485923 |
+----------+--------

{0: 0, 1: 1}
metric Euclidean
measure Accuracy
0.0030646324157714844
0.0020220279693603516
0.003990650177001953
0.0020232200622558594
+----------+----------+-----------------------+----------------------+----------------------+
|    k     | Accuracy |       Train Time      |      Test time       |      Total Time      |
+----------+----------+-----------------------+----------------------+----------------------+
|    5     |  0.9708  |     0:00:00.003065    |    0:00:00.589350    |    0:00:00.592415    |
|    15    |  0.9562  |     0:00:00.002022    |    0:00:00.535568    |    0:00:00.537590    |
|    30    |  0.9416  |     0:00:00.003991    |    0:00:00.615469    |    0:00:00.619459    |
|    45    |  0.9343  |     0:00:00.002023    |    0:00:00.599396    |    0:00:00.601420    |
| Average  |  0.9507  |     0:00:00.002775    |    0:00:00.584946    |    0:00:00.587721    |
| Std Dev. |  0.014   | 0.0008206448946469059 | 0.029991860802871525 | 0.030537415558059673 |
+----------+--------

...

{0: 0, 1: 1}
metric Euclidean
measure Accuracy
0.008465051651000977
0.006699800491333008
0.008237361907958984
0.0009996891021728516
+----------+----------+----------------------+---------------------+----------------------+
|    k     | Accuracy |      Train Time      |      Test time      |      Total Time      |
+----------+----------+----------------------+---------------------+----------------------+
|    5     |   0.55   |    0:00:00.008465    |    0:00:00.100715   |    0:00:00.109180    |
|    15    |   0.75   |    0:00:00.006700    |    0:00:00.109266   |    0:00:00.115966    |
|    30    |   0.75   |    0:00:00.008237    |    0:00:00.157071   |    0:00:00.165309    |
|    45    |   0.75   |    0:00:00.001000    |    0:00:00.109871   |    0:00:00.110871    |
| Average  |   0.7    |    0:00:00.006100    |    0:00:00.119231   |    0:00:00.125331    |
| Std Dev. |  0.0866  | 0.003022197719206045 | 0.02214527664182491 | 0.023215758852105337 |
+----------+----------+-----------------

{0: 0, 1: 1}
metric Euclidean
measure Accuracy
0.000997781753540039
0.001994609832763672
0.0019943714141845703
0.0019948482513427734
+----------+----------+------------------------+-----------------------+-----------------------+
|    k     | Accuracy |       Train Time       |       Test time       |       Total Time      |
+----------+----------+------------------------+-----------------------+-----------------------+
|    5     |   0.4    |     0:00:00.000998     |     0:00:00.103429    |     0:00:00.104427    |
|    15    |   0.75   |     0:00:00.001995     |     0:00:00.106808    |     0:00:00.108802    |
|    30    |   0.75   |     0:00:00.001994     |     0:00:00.112798    |     0:00:00.114792    |
|    45    |   0.75   |     0:00:00.001995     |     0:00:00.106908    |     0:00:00.108903    |
| Average  |  0.6625  |     0:00:00.001745     |     0:00:00.107486    |     0:00:00.109231    |
| Std Dev. |  0.1516  | 0.00043163925282964756 | 0.0033714814532669803 | 0.0036843345013903

{0: 0, 1: 1}
metric Euclidean
measure Accuracy
0.000997781753540039
0.0
0.0044672489166259766
0.00031280517578125
+----------+----------+-----------------------+----------------------+---------------------+
|    k     | Accuracy |       Train Time      |      Test time       |      Total Time     |
+----------+----------+-----------------------+----------------------+---------------------+
|    5     |  0.4211  |     0:00:00.000998    |    0:00:00.103469    |    0:00:00.104466   |
|    15    |  0.7895  |        0:00:00        |    0:00:00.119074    |    0:00:00.119074   |
|    30    |  0.7895  |     0:00:00.004467    |    0:00:00.110101    |    0:00:00.114568   |
|    45    |  0.7895  |     0:00:00.000313    |    0:00:00.108330    |    0:00:00.108643   |
| Average  |  0.6974  |     0:00:00.001444    |    0:00:00.110244    |    0:00:00.111688   |
| Std Dev. |  0.1595  | 0.0017821252497447148 | 0.005647172867793202 | 0.00557393541133181 |
+----------+----------+-----------------------+--

{0: 0, 1: 1}
metric Euclidean
measure Accuracy
0.0
0.0
0.0
0.001994609832763672
+----------+----------+-----------------------+----------------------+---------------------+
|    k     | Accuracy |       Train Time      |      Test time       |      Total Time     |
+----------+----------+-----------------------+----------------------+---------------------+
|    5     |  0.3684  |        0:00:00        |    0:00:00.107327    |    0:00:00.107327   |
|    15    |  0.7895  |        0:00:00        |    0:00:00.117365    |    0:00:00.117365   |
|    30    |  0.7895  |        0:00:00        |    0:00:00.132783    |    0:00:00.132783   |
|    45    |  0.7895  |     0:00:00.001995    |    0:00:00.118746    |    0:00:00.120741   |
| Average  |  0.6842  |     0:00:00.000499    |    0:00:00.119055    |    0:00:00.119554   |
| Std Dev. |  0.1823  | 0.0008636913929057853 | 0.009068562924398234 | 0.00909266296112423 |
+----------+----------+-----------------------+----------------------+-------------

{0: 0, 1: 1}
metric Euclidean
measure Accuracy
0.0
0.0
0.001995086669921875
0.001994609832763672
+----------+----------+-----------------------+---------------------+----------------------+
|    k     | Accuracy |       Train Time      |      Test time      |      Total Time      |
+----------+----------+-----------------------+---------------------+----------------------+
|    5     |  0.7895  |        0:00:00        |    0:00:00.108425   |    0:00:00.108425    |
|    15    |  0.7895  |        0:00:00        |    0:00:00.114493   |    0:00:00.114493    |
|    30    |  0.7895  |     0:00:00.001995    |    0:00:00.152646   |    0:00:00.154641    |
|    45    |  0.7895  |     0:00:00.001995    |    0:00:00.113162   |    0:00:00.115157    |
| Average  |  0.7895  |     0:00:00.000997    |    0:00:00.122182   |    0:00:00.123179    |
| Std Dev. |   0.0    | 0.0009974241399189412 | 0.01773244434353681 | 0.018352893572981335 |
+----------+----------+-----------------------+-------------------

In [2]:
"test"

'test'